<a href="https://colab.research.google.com/github/Shubh0405/VIThack20-Covid19Website/blob/Task4/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd;

#Load dataset and examine dataset, rename columns to questions and answers

df=pd.read_csv("NLP DATASET.csv");

df.columns=["questions","answers"];

df.head(10)

,questions,answers
0,What is a novel coronavirus?,A novel coronavirus is a new coronavirus that ...
1,Why is the disease being called coronavirus di...,"On February 11, 2020 the World Health Organiza..."
2,How does the virus spread?,The virus that causes COVID-19 is thought to s...
3,Can I get COVID-19 from food (including restau...,Currently there is no evidence that people can...
4,Will warm weather stop the outbreak of COVID-19?,It is not yet known whether weather and temper...
5,What is community spread?,Community spread means people have been infect...
6,Can mosquitoes or ticks spread the virus that ...,"At this time, CDC has no data to suggest that ..."
7,Does CDC recommend the use of masks to prevent...,Wear masks in public settings when around peop...
8,Is it safe to get care for my other medical co...,It is important to continue taking care of you...
9,"Am I at risk for COVID-19 from mail, packages,...",There is still a lot that is unknown about COV...


In [5]:
!pip install -U gensim
!pip install nltk

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [6]:
import re
import gensim 
from gensim.parsing.preprocessing import remove_stopwords

# from nltk.stem.lancaster import LancasterStemmer
# st = LancasterStemmer()

def clean_sentence(sentence, stopwords=False):
    sentence = str(sentence)
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    #sentence = re.sub(r'\s{2,}', ' ', sentence)
    
    if stopwords:
         sentence = remove_stopwords(sentence)
    
    # sent_stemmed='';
    # for word in sentence.split():
    #    sent_stemmed+=' '+st.stem(word) 
    # sentence=sent_stemmed
    
    return sentence

def get_cleaned_sentences(df,stopwords=False):    
    sents=df[["questions"]];
    cleaned_sentences=[]

    for index,row in df.iterrows():
        # print(index,row)
        cleaned=clean_sentence(row["questions"],stopwords);
        cleaned_sentences.append(cleaned);
    return cleaned_sentences;

cleaned_sentences=get_cleaned_sentences(df,stopwords=True)
print(cleaned_sentences);

print("\n")

cleaned_sentences_with_stopwords=get_cleaned_sentences(df,stopwords=False)
print(cleaned_sentences_with_stopwords);

['novel coronavirus', 'disease called coronavirus disease 2019 covid19', 'virus spread', 'covid19 food including restaurant takeout produce refrigerated packaged food drinking water', 'warm weather stop outbreak covid19', 'community spread', 'mosquitoes ticks spread virus causes covid19', 'cdc recommend use masks prevent covid19', 'safe care medical conditions time', 'risk covid19 mail packages products', 'okay donate blood', 'contact lens wearers special precautions prevent covid19', 'contact lens disinfecting solution eective covid19', 'use soap water hand sanitizer protect covid19', 'cleaning products use protect covid19', 'sick house gets sick', 'close contact covid19', 'risk child sick covid19', 'protect child covid19 infection', 'symptoms covid19 dierent children adults', 'children wear masks', 'prepare children case covid19 outbreak community', 'multisystem inammatory syndrome children misc risk', 'child hang friends', 'help child continue learning', 'kids access school meals', 

In [7]:
import numpy

sentences=cleaned_sentences_with_stopwords
#sentences=cleaned_sentences

# Split it by white space 
sentence_words = [[word for word in document.split() ]
         for document in sentences]

from gensim import corpora

dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
    print(key, ' : ', value)

import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow_corpus):
    print(sent)
    print(embedding)

question_orig=" what is coronavirus ?"
question=clean_sentence(question_orig,stopwords=False);
question_embedding = dictionary.doc2bow(question.split())


print("\n\n",question,"\n",question_embedding)

0  :  a
1  :  coronavirus
2  :  is
3  :  novel
4  :  what
5  :  2019
6  :  being
7  :  called
8  :  covid19
9  :  disease
10  :  the
11  :  why
12  :  does
13  :  how
14  :  spread
15  :  virus
16  :  can
17  :  drinking
18  :  food
19  :  from
20  :  get
21  :  i
22  :  including
23  :  or
24  :  packaged
25  :  produce
26  :  refrigerated
27  :  restaurant
28  :  takeout
29  :  water
30  :  of
31  :  outbreak
32  :  stop
33  :  warm
34  :  weather
35  :  will
36  :  community
37  :  causes
38  :  mosquitoes
39  :  that
40  :  ticks
41  :  cdc
42  :  masks
43  :  prevent
44  :  recommend
45  :  to
46  :  use
47  :  care
48  :  conditions
49  :  during
50  :  for
51  :  it
52  :  medical
53  :  my
54  :  other
55  :  safe
56  :  this
57  :  time
58  :  am
59  :  at
60  :  mail
61  :  packages
62  :  products
63  :  risk
64  :  blood
65  :  donate
66  :  me
67  :  okay
68  :  contact
69  :  lens
70  :  precautions
71  :  should
72  :  special
73  :  take
74  :  wearers
75  :  against
76

In [8]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity;
def retrieveAndPrintFAQAnswer(question_embedding,sentence_embeddings,FAQdf,sentences):
    max_sim=-1;
    index_sim=-1;
    for index,faq_embedding in enumerate(sentence_embeddings):
        #sim=cosine_similarity(embedding.reshape(1, -1),question_embedding.reshape(1, -1))[0][0];
        sim=cosine_similarity(faq_embedding,question_embedding)[0][0];
        print(index, sim, sentences[index])
        if sim>max_sim:
            max_sim=sim;
            index_sim=index;
       
    print("\n")
    print("Question: ",question)
    print("\n");
    print("Retrieved: ",FAQdf.iloc[index_sim,0]) 
    print(FAQdf.iloc[index_sim,1])        
    
retrieveAndPrintFAQAnswer(question_embedding,bow_corpus,df,sentences);

0 0.7071067811865475 what is a novel coronavirus
1 0.9999999999999998 why is the disease being called coronavirus disease 2019 covid19
2 0.773957299203321 how does the virus spread
3 0.7893522173763262 can i get covid19 from food including restaurant takeout produce refrigerated or packaged food or drinking water
4 0.7893522173763262 will warm weather stop the outbreak of covid19
5 0.9486832980505137 what is community spread
6 0.7893522173763262 can mosquitoes or ticks spread the virus that causes covid19
7 0.7893522173763262 does cdc recommend the use of masks to prevent covid19
8 0.9486832980505137 is it safe to get care for my other medical conditions during this time
9 0.7893522173763262 am i at risk for covid19 from mail packages or products
10 0.9486832980505137 is it okay for me to donate blood
11 0.7893522173763262 should contact lens wearers take special precautions to prevent covid19
12 0.9486832980505137 is contact lens disinfecting solution eective against covid19
13 0.7893

In [ ]:
from gensim.models import Word2Vec 
import gensim.downloader as api


glove_model=None;
try:
    glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
    print("Loaded glove model")
except:            
    glove_model = api.load('glove-twitter-25')
    glove_model.save("./glovemodel.mod")
    print("Saved glove model")
    
v2w_model=None;
try:
    v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
    print("Loaded w2v model")
except:            
    v2w_model = api.load('word2vec-google-news-300')
    v2w_model.save("./w2vecmodel.mod")
    print("Saved glove model")

w2vec_embedding_size=len(v2w_model['computer']);
glove_embedding_size=len(glove_model['computer']);

[==================================================] 100.0% 104.8/104.8MB downloaded
Saved glove model
[=================================-----------------] 67.1% 1116.3/1662.8MB downloaded

In [ ]:
def getWordVec(word,model):
        samp=model['computer'];
        vec=[0]*len(samp);
        try:
                vec=model[word];
        except:
                vec=[0]*len(samp);
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('computer', embeddingmodel);
        vec=numpy.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+numpy.array(getWordVec(word,embeddingmodel));
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [ ]:
#With w2Vec

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,v2w_model));

question_embedding=getPhraseEmbedding(question,v2w_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

In [ ]:
#With Glove

sent_embeddings=[];
for sent in cleaned_sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,glove_model));
    
question_embedding=getPhraseEmbedding(question,glove_model);

retrieveAndPrintFAQAnswer(question_embedding,sent_embeddings,df, cleaned_sentences);

In [ ]:
!pip install bert-serving-client

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()
res=bc.encode(['ML', 'AI'])

In [ ]:
question=clean_sentence(question_orig,stopwords=False);
cleaned_sentences=get_cleaned_sentences(df,stopwords=False)

sent_bertphrase_embeddings=[];
for sent in cleaned_sentences:
    sent_bertphrase_embeddings.append(bc.encode([sent]));
    
question_embedding=bc.encode([question]);

retrieveAndPrintFAQAnswer(question_embedding,sent_bertphrase_embeddings,df, cleaned_sentences);